# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 12 2022 12:27PM,248465,10,0086070501,ISDIN Corporation,Released
1,Oct 12 2022 12:27PM,248465,10,0086070499,ISDIN Corporation,Released
2,Oct 12 2022 12:27PM,248465,10,0086070500,ISDIN Corporation,Released
3,Oct 12 2022 12:27PM,248465,10,0086070502,ISDIN Corporation,Released
4,Oct 12 2022 12:27PM,248465,10,0086070504,ISDIN Corporation,Released
5,Oct 12 2022 12:27PM,248465,10,0086070505,ISDIN Corporation,Released
6,Oct 12 2022 12:27PM,248465,10,0086070517,ISDIN Corporation,Released
7,Oct 12 2022 12:27PM,248465,10,0086070518,ISDIN Corporation,Released
8,Oct 12 2022 12:27PM,248465,10,0086070528,ISDIN Corporation,Released
9,Oct 12 2022 12:27PM,248465,10,0086070529,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,248463,Released,7
31,248464,Released,2
32,248465,Released,76
33,248466,Released,5
34,248467,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248463,NaN,NaN,7.0
248464,NaN,NaN,2.0
248465,NaN,NaN,76.0
248466,NaN,NaN,5.0
248467,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248080,0.0,0.0,1.0
248265,0.0,20.0,10.0
248378,0.0,1.0,0.0
248381,10.0,12.0,4.0
248387,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248080,0,0,1
248265,0,20,10
248378,0,1,0
248381,10,12,4
248387,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248080,0,0,1
1,248265,0,20,10
2,248378,0,1,0
3,248381,10,12,4
4,248387,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248080,,,1
1,248265,,20,10
2,248378,,1,
3,248381,10,12,4
4,248387,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation
76,Oct 12 2022 12:26PM,248467,12,Hush Hush
77,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation
82,Oct 12 2022 12:18PM,248464,19,ACG North America LLC
84,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.
114,Oct 12 2022 12:16PM,248463,10,"Methapharm, Inc."
121,Oct 12 2022 12:12PM,248462,18,Innova Softgel LLC
122,Oct 12 2022 12:06PM,248458,10,Eywa Pharma Inc.
129,Oct 12 2022 12:05PM,248461,10,"Methapharm, Inc."
130,Oct 12 2022 12:03PM,248460,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation,,,76
1,Oct 12 2022 12:26PM,248467,12,Hush Hush,,,1
2,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation,,,5
3,Oct 12 2022 12:18PM,248464,19,ACG North America LLC,,,2
4,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.,,,30
5,Oct 12 2022 12:16PM,248463,10,"Methapharm, Inc.",,,7
6,Oct 12 2022 12:12PM,248462,18,Innova Softgel LLC,,,1
7,Oct 12 2022 12:06PM,248458,10,Eywa Pharma Inc.,,,7
8,Oct 12 2022 12:05PM,248461,10,"Methapharm, Inc.",,,1
9,Oct 12 2022 12:03PM,248460,10,"Methapharm, Inc.",,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation,76,,
1,Oct 12 2022 12:26PM,248467,12,Hush Hush,1,,
2,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation,5,,
3,Oct 12 2022 12:18PM,248464,19,ACG North America LLC,2,,
4,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.,30,,
5,Oct 12 2022 12:16PM,248463,10,"Methapharm, Inc.",7,,
6,Oct 12 2022 12:12PM,248462,18,Innova Softgel LLC,1,,
7,Oct 12 2022 12:06PM,248458,10,Eywa Pharma Inc.,7,,
8,Oct 12 2022 12:05PM,248461,10,"Methapharm, Inc.",1,,
9,Oct 12 2022 12:03PM,248460,10,"Methapharm, Inc.",19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation,76,,
1,Oct 12 2022 12:26PM,248467,12,Hush Hush,1,,
2,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation,5,,
3,Oct 12 2022 12:18PM,248464,19,ACG North America LLC,2,,
4,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.,30,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation,76.0,NaN,NaN
1,Oct 12 2022 12:26PM,248467,12,Hush Hush,1.0,NaN,NaN
2,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation,5.0,NaN,NaN
3,Oct 12 2022 12:18PM,248464,19,ACG North America LLC,2.0,NaN,NaN
4,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.,30.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 12:27PM,248465,10,ISDIN Corporation,76.0,0.0,0.0
1,Oct 12 2022 12:26PM,248467,12,Hush Hush,1.0,0.0,0.0
2,Oct 12 2022 12:23PM,248466,10,ISDIN Corporation,5.0,0.0,0.0
3,Oct 12 2022 12:18PM,248464,19,ACG North America LLC,2.0,0.0,0.0
4,Oct 12 2022 12:18PM,248459,10,Eywa Pharma Inc.,30.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3229821,162.0,12.0,10.0
12,745277,1.0,2.0,0.0
15,496691,11.0,63.0,2.0
16,248450,1.0,0.0,0.0
18,248462,1.0,0.0,0.0
19,1242208,6.0,11.0,0.0
21,744866,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3229821,162.0,12.0,10.0
1,12,745277,1.0,2.0,0.0
2,15,496691,11.0,63.0,2.0
3,16,248450,1.0,0.0,0.0
4,18,248462,1.0,0.0,0.0
5,19,1242208,6.0,11.0,0.0
6,21,744866,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,162.0,12.0,10.0
1,12,1.0,2.0,0.0
2,15,11.0,63.0,2.0
3,16,1.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,6.0,11.0,0.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,162.0
1,12,Released,1.0
2,15,Released,11.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21
Status,,,,,,,
Completed,10.0,0.0,2.0,0.0,0.0,0.0,0.0
Executing,12.0,2.0,63.0,0.0,0.0,11.0,0.0
Released,162.0,1.0,11.0,1.0,1.0,6.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21
0,Completed,10.0,0.0,2.0,0.0,0.0,0.0,0.0
1,Executing,12.0,2.0,63.0,0.0,0.0,11.0,0.0
2,Released,162.0,1.0,11.0,1.0,1.0,6.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21
0,Completed,10.0,0.0,2.0,0.0,0.0,0.0,0.0
1,Executing,12.0,2.0,63.0,0.0,0.0,11.0,0.0
2,Released,162.0,1.0,11.0,1.0,1.0,6.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()